In [2]:
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn.metrics as metrics
import numpy as np

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm


from data import ModelNet40, ScanObjectNN
from model import PointNet, DGCNN
from util import cal_loss, cal_loss_mix, IOStream
import gco
import plotly.graph_objects as go
import numpy as np

In [3]:
import argparse
args = argparse.Namespace(batch_size=30, data='MN40', dropout=0.5, emb_dims=1024, epochs=200, eval=False, exp_name='MultiSageMix', k=20, lr=0.0001, model='pointnet', model_path='', momentum=0.9, no_cuda=False, num_points=1024, seed=1, sigma=-1, test_batch_size=16, theta=0.2, use_sgd=False)

In [26]:
# Create an empty dictionary
id_label_dict = {}

# Open the file and read lines
with open('/home/ssawmya-local/CV/project/MultiPointmixup/data/modelnet40_ply_hdf5_2048/shape_names.txt', 'r') as file:  # replace 'filename.txt' with your file name
    lines = file.readlines()

# Populate the dictionary
for i, line in enumerate(lines):
    id_label_dict[i] = line.strip()  # remove any trailing newline characters

# Now, line_dict will be a dictionary where the key is the line number and the value is the line content
print(id_label_dict)

{0: 'airplane', 1: 'bathtub', 2: 'bed', 3: 'bench', 4: 'bookshelf', 5: 'bottle', 6: 'bowl', 7: 'car', 8: 'chair', 9: 'cone', 10: 'cup', 11: 'curtain', 12: 'desk', 13: 'door', 14: 'dresser', 15: 'flower_pot', 16: 'glass_box', 17: 'guitar', 18: 'keyboard', 19: 'lamp', 20: 'laptop', 21: 'mantel', 22: 'monitor', 23: 'night_stand', 24: 'person', 25: 'piano', 26: 'plant', 27: 'radio', 28: 'range_hood', 29: 'sink', 30: 'sofa', 31: 'stairs', 32: 'stool', 33: 'table', 34: 'tent', 35: 'toilet', 36: 'tv_stand', 37: 'vase', 38: 'wardrobe', 39: 'xbox'}


In [4]:
import pickle
direc = 'checkpoints/%s/data_save_for_viz_1' % args.exp_name
mixed = pickle.load(open(direc + "/data_dict_mix_3.p", "rb"))
non_mixed = pickle.load(open(direc + "/data_dict_normal_3.p", "rb"))


In [24]:
import plotly.subplots as sp
def plot_pointcloud(pc,label):
    point = pc

    print(label)

    # Define the 3D scatter plot
    # Define the 3D scatter plots
    # Create subplot with 2 rows and 1 column
    fig = sp.make_subplots(rows=1, cols=2)

    scatter1 = go.Scatter3d(
        x=point[:, 0],
        y=point[:, 1],
        z=point[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color='red',  # color set to red
            opacity=0.8
        ),
        name=label  # name displayed in legend
    )

    # give a title to the plot
    title = label
    
    # Define the layout of the plot with desired width and height and hide the axes
    layout = go.Layout(
        autosize=False,  # This enables custom size
        width=500,  # Custom width
        height=400,  # Custom height
        scene=dict(
            xaxis=dict(showticklabels=False, visible=False),
            yaxis=dict(showticklabels=False, visible=False),
            zaxis=dict(showticklabels=False, visible=False),
            # annotations=[  # add annotations
            #     dict(
            #         showarrow=False,
            #         x=point[0, 0],  # x position of the annotation
            #         y=point[0, 1],  # y position of the annotation
            #         z=point[0, 2],  # z position of the annotation
            #         text=label,  # text of the annotation
            #         xanchor="left",
            #         xshift=10,
            #         opacity=0.7
            #     )
            
            # ]
        )
    )

    # Create the plot
    fig = go.Figure(data=[scatter1], layout=layout)
    # fig.add_trace(scatter1, row=1, col=1)
    # fig.add_trace(scatter1, row=1, col=2)
    # Set the title
    fig.update_layout(title_text=title)
    # Show the plot
    fig.show()


In [25]:
pc = non_mixed['data'][0]
label = non_mixed['label'][0]
label_name = non_mixed['label_name'][0]
plot_pointcloud(pc,label_name)

flower_pot


In [33]:
for i in range(len(mixed['data'])):
    pc = mixed['data'][i]
    label = mixed['label'][i]
    mixed_id = label.nonzero()[0]
    if(len(mixed_id) == 2):
        label_name = "{:.2f} * {} + {:.2f} * {}".format(label[mixed_id[0]],id_label_dict[mixed_id[0]],label[mixed_id[1]],id_label_dict[mixed_id[1]])

    print(label_name)
    if(i == 3):
        plot_pointcloud(pc,label_name)
    
    # plot_pointcloud(pc,label_name)

0.00 * bookshelf + 1.00 * monitor
0.00 * chair + 1.00 * piano
1.00 * airplane + 0.00 * table
0.66 * desk + 0.34 * piano
0.66 * desk + 0.34 * piano


1.00 * car + 0.00 * desk
0.41 * bookshelf + 0.59 * keyboard
0.06 * car + 0.94 * sofa
0.24 * bed + 0.76 * toilet
0.00 * chair + 1.00 * flower_pot
0.00 * chair + 1.00 * flower_pot
0.77 * cone + 0.23 * plant
0.06 * chair + 0.94 * table
0.98 * bookshelf + 0.02 * wardrobe
0.59 * mantel + 0.41 * monitor
0.06 * chair + 0.94 * desk
0.81 * bookshelf + 0.19 * laptop
0.77 * chair + 0.23 * cone
0.17 * bookshelf + 0.83 * vase
0.00 * airplane + 1.00 * bed
0.04 * chair + 0.96 * laptop
0.69 * bed + 0.31 * vase
1.00 * sofa + 0.00 * toilet
0.87 * chair + 0.13 * mantel
0.90 * sofa + 0.10 * toilet
1.00 * bed + 0.00 * keyboard
0.47 * bed + 0.53 * chair
0.00 * bed + 1.00 * monitor
0.76 * bookshelf + 0.24 * monitor
0.90 * keyboard + 0.10 * plant
1.00 * cone + 0.00 * keyboard
1.00 * dresser + 0.00 * toilet
0.00 * bed + 1.00 * cone
0.10 * bed + 0.90 * wardrobe
0.99 * bed + 0.01 * toilet
0.00 * car + 1.00 * monitor
0.06 * sofa + 0.94 * tv_stand
0.42 * bed + 0.58 * toilet
1.00 * flower_pot + 0.00 * monitor
0.77 